# School Bus Scheduling

**Citation:** Zihao Xue, Xiangwen Deng, Bingxu Chen and Alaa Khamis, "School Bus Routing Using Metaheuristics Algorithms," IEEE International Conference on Smart Mobility, 2023.

 <b>Abstract:</b> Addressing school bus routing problem is important to ensure a safe and cost effective solution for students, parents and stakeholders. However, challenges in terms of multiple constraints and objectives are present. In this paper, school bus routing problem is formulated as contained multi-objective optimization problem. Cluster-first route-second scheme, genetic algorithm and adaptive genetic algorithm are applied to solve this problem. The performance of these algorithms are evaluated using real data of public schools in the City of Winchester, Virginia, USA. The conducted experiments showed that Cluster-first route-second yields the optimal solution.

## Problem Formulation

### Model assumption:
<p align="left">1.	All school buses depart from the school, and transport the students to school (single depot)</p>
<p align="left">2.	The coordinates of the school and stops and the number of shuttled students at each stop are determined.</p>
<p align="left">3.	The type of school bus is homogeneous.</p>
<p align="left">4.	Each school bus is not overloaded.</p>
<p align="left">5.	The waiting time at each stop is ignored.</p>
<p align="left">6.	Each stop is visited only once.</p>
<p align="left">7.	The length required to traverse any route does not exceed a maximum allowed policy.</p>
<p align="left">8. The route begins only when the buses reach their respective first stop, and ends when they ultimately reach school.</p>
<p align="left">9. All buses run at the same speed.</p>
<p align="left">10. There are more students than the capacity of one bus.</p><br>

### Variables:
$\textit{C}\mathrm{:\ The\ capacity\ of\ the\ bus}$ <br>

$\textit{N}\mathrm{:\ The\ number\ of\ school\ buses}$ <br>

$\textit{L}\mathrm{:\ The\ upper\ limit\ on\ the\ route\ length}$ <br>

$\textit{f}\mathrm{:\ The\ fixed\ unit\ cost\ to\ ship\ a\ school\ bus}$ <br>

$\alpha\mathrm{:\ The\ fixed\ unit\ cost\ of\ distance}$ <br>

$\textit{d}\mathrm{:\ The\ set\ of\ distance\ between\ sto}\mathrm{p}_i\mathrm{\ and\ sto}\mathrm{p}_j\mathrm{/school}$ <br>

$\textit{d}_{i,j}\mathrm{:\ The\ distance\ between\ sto}\mathrm{p}_i\mathrm{\ and\ sto}\mathrm{p}_j\mathrm{/school}$ <br>

$\textit{Std}\mathrm{:\ The\ set\ of\ students}$ <br>

$\textit{Std}_i\mathrm{\mathrm{:\ }The\ number\ of\ students\ at\ sto}\mathrm{p}_i\ $ <br>

$\textit{Stp}\mathrm{:\ The\ set\ of\ stops}$ <br>

$\textit{c}_k\mathrm{:\ Total\ cost\ for\ kth\ bus}$ <br>

$\textit{x}_{i,j,k}=\left\{ 
  \begin{array}{ c l }
    1, & \quad \textrm{if\ the\ kth\ bus\ traverses\ edge(i, j)} \\
    0,                 & \quad \textrm{otherwise}
  \end{array}
\right.$

$\textit{y}_{i,k}=\left\{ 
  \begin{array}{ c l }
    1, & \quad \textrm{if\ the\ kth\ bus\ visted\ sto}\mathrm{p}_i\} \\
    0,                 & \quad \textrm{otherwise}
  \end{array}
\right.$

<br><br>

### Objective function:
$\min{\sum_{k=1}^{N}c_k}$ <br>

$c_k=f+\sum_{i\in S t p}\sum_{j\in S t p}{\alpha\ast d_{i,j}\ast x_{i,j,k}}$ <br>

### Subject function:
$\sum_{i\in S t d}{Std_i\ast y_{i,k}\le C}\ where\ k\in\left[1,N\right]$ <br>

$\sum_{k=1}^{N}{\sum_{i\in S t p}{Stp_i}\ast y_{i,k}}=Size\left(Stp\right)$ <br>

$\sum_{k=1}^{N}y_{i,k}=1\ where\ i\in Stp$ <br>

$\sum_{i\in S t p}\sum_{j\in S t p}{d_{i,j}\ast x_{i,j,k}}\le L\ where\ k\in\left[1,N\right]$ <br>


## Dataset
The depots dataset is compiled from [The City of Winchester's OPEN DATA PORTAL](https://city-of-winchester-open-data-1-winchestercity.hub.arcgis.com/datasets/winchestercity::school-bus-stops/about).
As the number of students in each school bus stop is not available in public, we compile the number of students by ourselves.
The total number of students in QES is 454 which can be found in public.
As the capacity of a school bus for QES is not available in public, we assume the capacity of a school bus is 72 children.
All the required information about the coordinates of school bus stops for QES, the number of students in each school bus stop
and the capacity of a school bus are included in the dataset.

The trip matrix dataset is pre-calculated matrix of distances and routes between depots based on Dijkstra Search.

In [1]:
import pandas as pd
import math
import numpy as np
import random
import time
import osmnx
import heapq
import folium
import folium.plugins as plugins
import copy


# ACS_TSP is the same as ACS class
# As jupyter can't run multiprocessing directly
# We import py file to do the multiprocessing
import mTSP
import multiprocessing
import ACS_TSP

### Read data

In [2]:
# Import location data
depots = pd.read_csv('data/Final_School_Bus_Stops_QES.csv')

# Import a pre-calculated trip distance matrix
# These distances are generated based on Dijkstra Search
trip_matrix = pd.read_csv('data/Trip_matrix.csv')

### Solvers

### Genetic Algorithm

In [3]:
class BusStop(object):
    def __init__(self,
                 id: int,
                 students: int):
        """

        :param id: id of the stop
        :param students: students at the stop
        """
        self.stop_id = id
        self.students = students

In [4]:
class BusRoute(object):
    def __init__(self, bus: int):
        """
        initialize a route for current bus

        :param bus: the id of the bus
        """
        self.bus = bus
        self.stops = []  # stops traversed in this bus route (idx is the order)
        self.total_distance = 0.0  # total distance traveled in this bus route
        self.curr_students = 0  # students in this bus route

    def assign_with_new_stop(self,
                             stop: BusStop,
                             distance_metric):
        """

        :param stop: new stop to be assigned
        :param distance_matrix: distance matrix
        :return: None
        """

        stops_copied = copy.deepcopy(self.stops)
        stops_copied.append(stop)

        # start from the stop with the furthest distance from school
        max_dist_from_school = math.inf
        first_stop_idx = 0
        for i in range(len(stops_copied)):
            stop_id = stops_copied[i].stop_id
            if distance_metric[0][stop_id] < max_dist_from_school:
                max_dist_from_school = distance_metric[0][stop_id]
                first_stop_idx = i
        max_dist_from_school = distance_metric[0][stops_copied[first_stop_idx].stop_id]
        assigned_stops = [stops_copied[first_stop_idx]]
        total_students = stops_copied[first_stop_idx].students
        stops_copied.remove(stops_copied[first_stop_idx])
        total_distance = max_dist_from_school

        # a greedy way to update the order of the stops
        while len(stops_copied) > 0:
            last_stop_id = assigned_stops[-1].stop_id
            min_dist_from_last = math.inf
            idx_chosen = 0

            # find the nearest stop from current stop
            for i in range(len(stops_copied)):
                stop_id = stops_copied[i].stop_id
                if distance_metric[last_stop_id][stop_id] < min_dist_from_last:
                    min_dist_from_last = distance_metric[last_stop_id][stop_id]
                    idx_chosen = i

            # assign the stop in order
            assigned_stops.append(stops_copied[idx_chosen])
            total_students += stops_copied[idx_chosen].students
            total_distance += min_dist_from_last
            stops_copied.remove(stops_copied[idx_chosen])

        # back to school
        last_stop_id = assigned_stops[-1].stop_id
        total_distance += distance_metric[last_stop_id][0]
        self.stops = assigned_stops
        self.curr_students = total_students
        self.total_distance = total_distance

In [5]:
class Individual(object):
    def __init__(self, stops, distance_metric, buses_available):
        """
        individual
        """
        self.stops = stops
        self.distance_metric = distance_metric
        self.routes = [BusRoute(bus) for bus in range(buses_available)]  # routes for each bus
        self.buses_available = buses_available
        self.gene = [[0, 0] for _ in range(
            len(stops))]  # gene for current individual (2-index for each stop: 0: bus idx, 1: order)
        self.fitness = 0.0  # fitness for the individual
        self.traversed_distance = 0.0  # total distance traveled in all routes


    def assign_stop(self,
                    bus_idx: int,
                    stop: BusStop,
                    capacity: int
                    ):
        """

        :param bus_idx:  bus idx to assign
        :param stop:  stop to assign
        :param capacity: capacity of a single bus
        :return: True if successfully assigned else False
        """

        route = self.routes[bus_idx]

        # the bus should not be overloaded
        if route.curr_students + stop.students > capacity:
            return False

        # assign the stop to bus
        route.assign_with_new_stop(stop, self.distance_metric)
        return True

    def encode_to_gene(self):
        """
        encode routes to gene
        :return: None
        """
        for route in self.routes:
            bus = route.bus
            stops = route.stops
            # set gene according to the bus idx and stop order
            for i in range(len(stops)):
                stop = stops[i]
                self.gene[stop.stop_id - 1][0] = bus
                self.gene[stop.stop_id - 1][1] = i

    def decode_from_gene(self, from_gene):
        """
        decode gene to routes
        :param from_gene: gene
        :return: None
        """
        # initialize routes
        routes = [BusRoute(bus) for bus in range(0, self.buses_available)]
        bus_dict = {}

        # store the order of stops in each route
        for i in range(len(from_gene)):
            segment = from_gene[i]
            bus = segment[0]
            v = bus_dict.get(bus, [])
            heapq.heappush(v, (segment[1], i))
            bus_dict[bus] = v

        # assign stops to routes based on the order
        for bus, v in bus_dict.items():
            last_stop_id = 0
            while len(v) > 0:
                next_stop_idx = heapq.heappop(v)[1]
                next_stop = self.stops[next_stop_idx]
                next_stop_id = next_stop.stop_id
                routes[bus].stops.append(next_stop)
                routes[bus].curr_students += next_stop.students
                routes[bus].total_distance += self.distance_metric[last_stop_id][next_stop_id]
                last_stop_id = next_stop_id

            # back to school
            routes[bus].total_distance += self.distance_metric[last_stop_id][0]

        self.routes = routes
        # regenerate the gene
        self.encode_to_gene()

    def calc_and_set_fitness(self, max_distance, distance_penalty, bus_unit_cost):
        """
        calculate fitness
        penalties are:
            1. number of buses (less is better)
            2. distance of single route (less is better)
        :param max_distance: max distance a bus can travel
        :param distance_penalty: penalty if exceed max distance
        :param unit cost to run a single bus
        :return: None
        """
        distance_all = 0.0
        buses_consumed = 0
        fitness_score = 0

        for route in self.routes:
            if len(route.stops) > 0:
                distance_all += route.total_distance
                # for distance that exceeds max distance, add penalty to its fitness
                if route.total_distance > max_distance:
                    fitness_score += max_distance + (
                            route.total_distance - max_distance) * distance_penalty
                else:
                    fitness_score += route.total_distance
                buses_consumed += 1

        self.traversed_distance = distance_all

        # calculate fitness
        self.fitness = 1000000 / (buses_consumed * bus_unit_cost + fitness_score)

    def adapt(self, capacity):
        """
        individual should adapt to the environment conditions (every bus should not be overloaded)
        :param capacity: capacity of a single bus
        :return: None
        """
        running_routes = []  # routes that have stops assigned into (bus is running)
        empty_routes = []  # routes with no stops (bus is not running)
        unassigned_stops = []
        for route in self.routes:
            students_in_route = 0
            if len(route.stops) == 0:
                empty_routes.append(route)
                continue
            running_routes.append(route)
            unassigned_start_i = -1
            new_route_distance = 0
            last_stop_id = 0
            # for each route, keep the order of its stops
            for i in range(len(route.stops)):
                stop = route.stops[i]
                if students_in_route + stop.students <= capacity:
                    # capacity satisfied then re-calculate distance and students number
                    new_route_distance += self.distance_metric[last_stop_id][stop.stop_id]
                    students_in_route += stop.students
                    last_stop_id = stop.stop_id
                else:
                    # set the start idx of the stops that exceeds the capacity
                    unassigned_start_i = i
                    break

            route.total_distance = new_route_distance
            route.curr_students = students_in_route

            # unassign the subsequent stops from the route and cache these unassigned stops
            if unassigned_start_i > -1:
                for j in range(unassigned_start_i, len(route.stops)):
                    stop = route.stops[j]
                    unassigned_stops.append(stop)
                route.stops = route.stops[:unassigned_start_i]

        # re-assign the unassigned stops
        for stop in unassigned_stops:
            assigned = False
            # prioritize from running routes to assign
            for running_route in running_routes:
                if running_route.curr_students + stop.students < capacity:
                    running_route.curr_students += stop.students
                    running_route.total_distance += self.distance_metric[running_route.stops[-1].stop_id][stop.stop_id]
                    running_route.stops.append(stop)
                    assigned = True
                    break

            # if not assigned, assign the stop to an empty route
            if not assigned:
                empty_route = empty_routes.pop()
                empty_route.stops.append(stop)
                empty_route.curr_students += stop.students
                empty_route.total_distance += self.distance_metric[empty_route.stops[-1].stop_id][stop.stop_id]
                # since the stop is assigned to, the empty route then becomes a running route
                running_routes.append(empty_route)

        # back to school
        for route in running_routes:
            last_stop_id = route.stops[-1].stop_id
            route.total_distance += self.distance_metric[last_stop_id][0]

        self.encode_to_gene()

In [6]:
class Generation(object):
    def __init__(self, population_size, selection_rate, elites_rate):
        """
        generation that contains a population of different individuals
        :param population_size: number of individuals in a population
        :param selection_rate: ratio of selection
        :param elites_rate: ratio of elites
        """
        self.population = []
        self.selected_population = []  # can be chosen to crossover
        self.elite_population = []  # will survive to the next generation
        self.total_fitness = 0.0  # sum of fitness of every individual in population
        self.selected_avg = 0.0
        self.population_size = population_size
        self.selection_rate = selection_rate
        self.elites_rate = elites_rate
        self.dominated_individual = None

    def add_individual(self, individual: Individual):
        """
        add individual to population
        :param individual: individual to be added to current population
        :return: None
        """
        if not self.dominated_individual:
            self.dominated_individual = individual

        if individual.fitness > self.dominated_individual.fitness:
            self.dominated_individual = individual

        self.population.append(individual)
        self.total_fitness += individual.fitness

    def select(self):
        """
        set the selected population qualified for crossover
        :return: None
        """
        self.population.sort(reverse=True,
                             key=lambda individual: individual.fitness)
        selected_count = round(self.selection_rate * self.population_size)
        self.selected_population = self.population[:selected_count]
        self.selected_avg = sum([individual.fitness for individual in self.selected_population]) / selected_count

    def select_parents(self):
        """
        choose parents from selected population for crossover with a probability according to their fitness
        :return: parents
        """
        parents = random.choices(self.selected_population,
                                 weights=[individual.fitness for individual in self.selected_population],
                                 k=2)
        return parents[0], parents[1]

    def select_individual(self):
        """
        randomly select one individual from the population (often used for mutation)
        :return: selected individual
        """
        return random.choices(self.selected_population, k=1)[0]

    def merge_elites(self, elites_from_last_generation):
        """
        the elites from last generation will take the places of  most inferior individuals from current generation
        :param elites_from_last_generation
        :return: None
        """
        # merge the population
        self.population += elites_from_last_generation

        # add up to total fitness
        self.total_fitness += sum([individual.fitness for individual in elites_from_last_generation])

        # sort by fitness
        self.population.sort(reverse=True,
                             key=lambda individual: individual.fitness)

        # eliminate the inferior individuals and adjust the total fitness
        self.total_fitness -= sum([individual.fitness for individual in self.population[-len(elites_from_last_generation):]])
        self.population = self.population[:self.population_size]
        elites_count = round(self.elites_rate * self.population_size)

        # re-generate the elite population
        self.elite_population = self.population[:elites_count]
        self.dominated_individual = self.elite_population[0]

    def get_elites(self):
        """
        get elites from current generation
        :return: elites
        """
        if len(self.elite_population) > 0:
            return self.elite_population

        # sort individuals accoring to their fitness
        self.population.sort(reverse=True,
                             key=lambda individual: individual.fitness)
        elites_count = round(self.elites_rate * self.population_size)

        # generate the elite population
        self.elite_population = self.population[:elites_count]
        return self.elite_population

In [7]:
def load_distance_metric(df_routes):
    distance_metric = [[0.0 for _ in range(len(df_routes.i.unique()))] for _ in range(len(df_routes.j.unique()))]

    for row in df_routes.iterrows():
        i = int(row[1][2])
        j = int(row[1][3])
        dist = row[1][4]
        distance_metric[i][j] = dist

    return distance_metric


def load_bus_stops(df_stops):
    stops = []
    for id in range(1, len(df_stops)):
        stop = BusStop(id, df_stops.loc[id, "NUMBER OF STUDENTS"])
        stops.append(stop)
    return stops

def exp_schedule(k=20, lam=0.005, limit=100):
    function = lambda t: k - (k * math.exp(-lam * t) if t < limit else 0)
    return function

In [8]:
class GA(object):
    def __init__(self, population_size,
                 crossover_rate,
                 mutation_rate,
                 selection_rate,
                 elites_rate,
                 stops,
                 distance_metric,
                 params):
        """

        :param crossover_rate: rate of crossover
        :param mutation_rate: rate of mutation
        :param adaptive_function: function for adaptive mutation
        """
        self.generation = Generation(population_size, selection_rate, elites_rate)  # current generation (population)
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.initial_mutation_rate = mutation_rate
        self.selection_rate = selection_rate
        self.initial_selection_rate = selection_rate
        self.elites_rate = elites_rate
        self.stops = stops
        self.distance_metric = distance_metric
        self.params = params

    def crossover(self,
                  parent1: Individual,
                  parent2: Individual):
        """
        crossover function
        :param parent1: crossover parent1
        :param parent2: crossover parent2
        :return: child after crossover
        """

        # find two edges of crossover segment
        cross_idx1 = random.randint(0, len(parent1.gene) - 1)
        cross_idx2 = random.randint(cross_idx1 + 1, len(parent1.gene))

        child1 = Individual(self.stops, self.distance_metric, self.params['buses_available'])
        child2 = Individual(self.stops, self.distance_metric, self.params['buses_available'])
        child1_gene = child1.gene
        child2_gene = child2.gene

        for i in range(0, cross_idx1):
            child1_gene[i][0] = parent1.gene[i][0]
            child1_gene[i][1] = parent1.gene[i][1]
            child2_gene[i][0] = parent2.gene[i][0]
            child2_gene[i][1] = parent2.gene[i][1]

        # crossover between the crossover points
        for i in range(cross_idx1, cross_idx2):
            child1_gene[i][0] = parent2.gene[i][0]
            child1_gene[i][1] = parent2.gene[i][1]
            child2_gene[i][0] = parent1.gene[i][0]
            child2_gene[i][1] = parent1.gene[i][1]

        for i in range(cross_idx2, len(parent1.gene)):
            child1_gene[i][0] = parent1.gene[i][0]
            child1_gene[i][1] = parent1.gene[i][1]
            child2_gene[i][0] = parent2.gene[i][0]
            child2_gene[i][1] = parent2.gene[i][1]

        # decode from gene
        child1.decode_from_gene(child1_gene)
        child2.decode_from_gene(child2_gene)

        # the child with higher fitness would survive
        child1.calc_and_set_fitness(self.params['max_distance'], self.params['distance_penalty'], self.params['bus_unit_cost'])
        child2.calc_and_set_fitness(self.params['max_distance'], self.params['distance_penalty'], self.params['bus_unit_cost'])

        return child1 if child1.fitness > child2.fitness else child2

    def mutation(self, individual: Individual):
        """

        :param individual: individual to mutate
        :return: mutated individual
        """
        gene = individual.gene
        mutated_genes = [copy.deepcopy(gene) for _ in range(5)]

        # randomly choose three indices from gene
        mutation_indices = random.sample([i for i in range(len(gene))], 3)

        mutation_locs = [[0, 2, 1], [1, 2, 0], [1, 0, 2], [2, 0, 1], [2, 1, 0]]

        mutated_individuals = [Individual(self.stops, self.distance_metric, self.params['buses_available']) for _ in range(len(mutated_genes))]

        outstanding_fitness = individual.fitness
        successful_mutation = individual

        # generate five different mutations
        for i in range(len(mutated_genes)):
            for j in range(3):
                mutated_gene = mutated_genes[i]
                curr_mutate_index_from = mutation_indices[j]
                curr_mutate_index_to = mutation_indices[mutation_locs[i][j]]
                # exchange the positions of gene among three indices
                mutated_gene[curr_mutate_index_from][0] = gene[curr_mutate_index_to][0]
                mutated_gene[curr_mutate_index_from][1] = gene[curr_mutate_index_to][1]

            # decode from gene
            mutated_individuals[i].decode_from_gene(mutated_genes[i])
            mutated_individuals[i].calc_and_set_fitness(self.params['max_distance'], self.params['distance_penalty'], self.params['bus_unit_cost'])
            if mutated_individuals[i].fitness > outstanding_fitness:
                successful_mutation = mutated_individuals[i]

        # if there has been a successful mutation, it survives
        if successful_mutation != individual:
            return successful_mutation

        mutated_individuals.append(individual)

        # else choose among the mutations (including original one) according to there fitness
        return random.choices(mutated_individuals,
                              weights=[individual.fitness for individual in mutated_individuals],
                              k=1)[0]

    def generate_initial_population(self):
        """
        generate the initial population of GA
        """


        buses = [i for i in range(self.params['buses_available'])]

        for i in range(self.population_size):
            unassigned_stops = copy.deepcopy(self.stops)

            # randomly choose a bus
            new_bus = buses[random.randint(0, len(buses) - 1)]
            individual = Individual(self.stops, self.distance_metric, self.params['buses_available'])
            # assign all stops
            while len(unassigned_stops) > 0:
                stop_idx = random.randint(0, len(unassigned_stops) - 1)

                stop_to_assign = unassigned_stops[stop_idx]
                unassigned_stops.remove(unassigned_stops[stop_idx])

                # ensure assigned
                while not individual.assign_stop(new_bus, stop_to_assign, params['capacity']):
                    # if failed, randomly choose a new bus
                    new_bus = buses[random.randint(0, len(buses) - 1)]

            # encode the individual to gene
            individual.encode_to_gene()
            individual.calc_and_set_fitness(params['max_distance'], params['distance_penalty'], params['bus_unit_cost'])
            self.generation.add_individual(individual)

    def adaptive_mutation_rate(self, individual: Individual):
        """
        different individuals may have different chances for mutations
        a powerful individual has lower chance for mutation (higher chance to survive)
        :param individual:
        :return: adaptive mutation rate
        """
        # return self.mutation_rate

        if individual.fitness < self.generation.selected_avg:
            return self.mutation_rate
        else:
            return self.mutation_rate * (0.1 + 0.9 * (1 if self.generation.dominated_individual.fitness == self.generation.selected_avg else (self.generation.population[0].fitness - individual.fitness) / (
                    self.generation.population[0].fitness - self.generation.selected_avg)))

    def run_GA(self, generations):
        """
        function to run the GA process
        :param generations: number of iterations
        :return:  solution population
        """
        self.generate_initial_population()

        for iter in range(generations):
            # select individuals that are qualified for choosing to crossover
            self.generation.select()
            next_generation = Generation(self.population_size, self.selection_rate, self.elites_rate)
            while len(next_generation.population) < self.population_size:
                if random.random() < self.crossover_rate:
                    # select two parents for crossover
                    parent1, parent2 = self.generation.select_parents()
                    # do the crossover
                    child = self.crossover(parent1, parent2)
                else:
                    # select one individual
                    individual = self.generation.select_individual()
                    child = Individual(self.stops, self.distance_metric, self.params['buses_available'])
                    child.decode_from_gene(individual.gene)

                if random.random() < self.mutation_rate:
                    # do the mutation
                    child = self.mutation(child)

                # adapt the child to environment conditions
                child.adapt(self.params['capacity'])
                child.calc_and_set_fitness(params['max_distance'], params['distance_penalty'], params['bus_unit_cost'])
                next_generation.add_individual(child)

            # print("Generation {0} with fitness: {1} and total_distance: {2}".format(iter + 1,
            #                                                                         next_generation.dominated_individual.fitness,
            #                                                                         next_generation.dominated_individual.traversed_distance))
            self.generation = next_generation

        best_individual = self.generation.dominated_individual
        best_routes = []

        for route in best_individual.routes:

            if len(route.stops) > 0:
                route_arr = [0]
                for stop in route.stops:
                    route_arr.append(stop.stop_id)
                best_routes.append(route_arr)

        return best_individual.traversed_distance, best_routes

    def run_adaptive_GA(self, generations, adaptive_function):
        """
        function to run the adaptive version of GA process
        :param generations: number of iterations
        :param adaptive_function: function that helps to generate adaptive mutation rate
        :return:  solution population
        """
        self.generate_initial_population()

        for iter in range(generations):
            # select individuals that are qualified for choosing to crossover
            self.generation.select()
            next_generation = Generation(self.population_size, self.selection_rate, self.elites_rate)
            while len(next_generation.population) < self.population_size:
                if random.random() < self.crossover_rate:
                    # select two parents for crossover
                    parent1, parent2 = self.generation.select_parents()
                    # do the crossover
                    child = self.crossover(parent1, parent2)
                else:
                    # select one individual
                    individual = self.generation.select_individual()
                    child = Individual(self.stops, self.distance_metric, self.params['buses_available'])
                    child.decode_from_gene(individual.gene)

                if random.random() < self.adaptive_mutation_rate(child):
                    # do the mutation
                    child = self.mutation(child)

                # adapt the child to environment conditions
                child.adapt(self.params['capacity'])
                child.calc_and_set_fitness(params['max_distance'], params['distance_penalty'], params['bus_unit_cost'])
                next_generation.add_individual(child)

            # elites from last generation would survive
            elites_from_last_generation = self.generation.get_elites()

            if next_generation.dominated_individual.fitness < self.generation.dominated_individual.fitness:
                next_generation.merge_elites(elites_from_last_generation)

            adaptive_param = (next_generation.dominated_individual.fitness - next_generation.total_fitness / self.population_size) / next_generation.dominated_individual.fitness
            self.mutation_rate = adaptive_function(1/adaptive_param)

            # print("Generation {0} with fitness: {1} and total_distance: {2}".format(iter + 1,
            #                                                                         next_generation.dominated_individual.fitness,
            #                                                                         next_generation.dominated_individual.traversed_distance))
            self.generation = next_generation

        best_individual = self.generation.dominated_individual
        best_routes = []

        for route in best_individual.routes:

            if len(route.stops) > 0:
                route_arr = [0]
                for stop in route.stops:
                    route_arr.append(stop.stop_id)
                best_routes.append(route_arr)

        return best_individual.traversed_distance, best_routes

#### Cluster-First Route-Second ( Sweep Nearest Algorithm + Ant Colony System)

#### Ant Colony System

In [9]:
class ACS:
    class Edge:
        def __init__(self, a, b, dist, initial_pheromone):
            self.a = a
            self.b = b
            self.dist = dist
            self.pheromone = initial_pheromone

    class Ant:
        def __init__(self, alpha, beta, num_nodes, edges, r0=0.9):
            self.alpha = alpha
            self.beta = beta
            self.num_nodes = num_nodes
            self.edges = edges
            self.r0 = r0
            self.tour = None
            self.distance = 0.0

        # a function to calculate pheremone levels
        def pheremone(self, level, distance):
            return level ** self.alpha * ((1.0/distance)) ** self.beta

        def select_next_stop(self):

            unvisited_nodes = [stop for stop in range(self.num_nodes) if stop not in self.tour]
            children_pheremones = []
            transition_sum = 0.0

            for stop in unvisited_nodes:
                children_pheremones.append(
                    self.pheremone(
                        self.edges[self.tour[-1]][stop].pheromone,
                        self.edges[self.tour[-1]][stop].dist,
                    )
                )

            random_value = random.uniform(0, 1)

            # when r <= r0, next stop = argmax(children_pheremones)
            if random_value <= self.r0:
                return unvisited_nodes[children_pheremones.index(max(children_pheremones))]

            transition_sum = sum(children_pheremones)

            transition_probability = [
                children_pheremones[i] / transition_sum
                for i in range(len(children_pheremones))
            ]

            # Probabilistically choose a child to explore based weighted by transition probability
            chosen = random.choices(unvisited_nodes, weights=transition_probability, k=1)[0]
            return chosen

        def get_tour(self):
            # start from school
            self.tour = [0]
            for i in range(self.num_nodes - 1):
                self.tour.append(self.select_next_stop())
            return self.tour

        def get_distance(self):
            self.distance = 0.0
            for i in range(self.num_nodes):
                self.distance += self.edges[self.tour[i]][self.tour[(i + 1) % self.num_nodes]].dist
            return self.distance

    def __init__(self, stops, total_num_nodes, colony_size=10, alpha=1.0, beta=3.0, r0 = 0.5,
                 rho=0.1, pheromone_deposit_weight=1.0, initial_pheromone=1.0, iteration=300):
        self.colony_size = colony_size
        self.rho = rho
        self.pheromone_deposit_weight = pheromone_deposit_weight
        self.iteration = iteration
        self.total_num_nodes = total_num_nodes
        self.global_best_tour = None
        self.actual_global_best_tour = None
        self.initial_pheromone = initial_pheromone

        self.global_best_distance = float("inf")
        self.alpha = alpha
        self.beta = beta
        self.r0 = r0
        self.edge_list = None

        self.total_edges = [[None] * self.total_num_nodes for _ in range(self.total_num_nodes)]
        self.edges = []
        self.num_nodes = 0

        for index, row in stops.iterrows():
            self.total_edges[row['i']][row['j']] = self.Edge(row['i'],row['j'], row['dist'] , initial_pheromone)

    def reset_sub_edges(self, edge_list):
        self.edge_list = edge_list
        self.num_nodes = len(edge_list)
        self.edges = [[None] * self.num_nodes for _ in range(self.num_nodes)]
        self.global_best_distance = float("inf")
        self.global_best_tour = None
        self.actual_global_best_tour = None

        for i in range(len(edge_list)):
            for j in range(i, len(edge_list)):
                edge = self.total_edges[edge_list[i]][edge_list[j]]
                edge.pheromone = self.initial_pheromone
                self.edges[i][j] = self.edges[j][i] = edge

    def evaporate_pheromone(self, evaporate):
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                self.edges[i][j].pheromone *= evaporate

    def update_pheromone(self, tour, distance, weight=1.0):
        evaporate = (1 - self.rho)
        self.evaporate_pheromone(evaporate)

        pheromone_to_add = self.pheromone_deposit_weight / distance
        for i in range(self.num_nodes):
            self.edges[tour[i]][tour[(i + 1) % self.num_nodes]].pheromone += weight * pheromone_to_add

    def start(self):
        if(self.num_nodes == 1):
            self.actual_global_best_tour = self.edge_list
            self.global_best_distance = self.edges[0][self.edge_list[0]].dist
            return

        for iter in range(self.iteration):
            local_best_tour = None
            local_best_dist = float("inf")
            for step in range(self.colony_size):
                ant = self.Ant(self.alpha, self.beta, self.num_nodes, self.edges, self.r0)

                # local update pheromone
                cur_tour = ant.get_tour()
                cur_dist = ant.get_distance()
                self.update_pheromone(cur_tour, cur_dist)

                if ant.distance < local_best_dist:
                    local_best_tour = ant.tour
                    local_best_dist = ant.distance
            self.update_pheromone(local_best_tour, local_best_dist)

            if local_best_dist < self.global_best_distance:
                self.global_best_distance = local_best_dist
                self.global_best_tour = local_best_tour

            # global update pheromone
            self.update_pheromone(self.global_best_tour, self.global_best_distance)

        self.convert_to_actual_tour()

    def convert_to_actual_tour(self):
        self.actual_global_best_tour = []
        for i in self.global_best_tour:
            self.actual_global_best_tour.append(self.edge_list[i])

#### Sweep Nearest Algorithm

In [10]:
class Sweep:
    class Stop:
        def __init__(self, coordinate, student_num, label, depot):
            self.label = label
            self.coordinate = coordinate
            self.angle = self.get_angle(coordinate[0], coordinate[1], depot[0], depot[1])
            self.student_num = student_num

        def get_angle(slef, lat1, lon1, lat2, lon2):
            dLon = lon2 - lon1;
            y = math.sin(dLon) * math.cos(lat2);
            x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon);
            brng = np.rad2deg(math.atan2(y, x));
            if brng < 0: brng += 360
            return brng

    class Edge:
        def __init__(self, label):
            self.dist_list = []
            self.label = label
            self.label_list = []
            self.sorted_label_list = []
            self.sorted_dist_list = []
            self.dist_dict = dict()

        def sort(self):
            dist = np.array(self.dist_list)
            dist_arg = dist.argsort()
            self.sorted_dist_list = [self.dist_list[i] for i in dist_arg]
            self.sorted_label_list = [self.label_list[i] for i in dist_arg]

    def __init__(self, depots, dist, capacity, maxing_riding_dist, TSP_solver, start_index):
        self.depots = depots
        self.dist = dist
        self.stops = []
        self.depot = []
        self.number_nodes = len(depots.index)
        self.edges = [None] * self.number_nodes
        self.stops_dict = dict()
        self.capacity = capacity
        self.maxing_riding_dist = maxing_riding_dist
        self.TSP_solver = TSP_solver
        self.start_index = start_index

        for index, row in depots.iterrows():
            if index == 0:
                self.depot = [row['lat'], row['lng']]
            else:
                self.stops.append(self.Stop([row['lat'], row['lng']], row['NUMBER OF STUDENTS'], index, self.depot))

        for index, row in self.dist.iterrows():
            if self.edges[row['i']] is None:
                self.edges[row['i']] = self.Edge(row['i'])
            if row['i'] == 0 or row['j'] != 0:
                self.edges[row['i']].dist_list.append(row['dist'])
                self.edges[row['i']].label_list.append(row['j'])
                self.edges[row['i']].dist_dict[row['j']] = row['dist']

        # sort dist list in edge
        for edge in self.edges:
            edge.sort()

        # save stop into dict
        for stop in self.stops:
            self.stops_dict[stop.label] = stop

        angles_arg = self.sort_by_angle()
        self.sorted_stops = [self.stops[i] for i in angles_arg]

    def sort_by_angle(self):
        angles = np.array([stop.angle for stop in self.stops])
        angles_arg = angles.argsort()
        return angles_arg

    def next_nearest_stop_label(self, visted_stop, cur_stop):
        edge = self.edges[cur_stop]
        for next in edge.sorted_label_list:
            if next not in visted_stop:
                return next
        return None

    def reverse_sorted_stops(self):
        self.sorted_stops.reverse()

    def get_clusters(self):
        cur_cluster = []
        clusters = []
        visted_stop = set()
        cur_capacity = 0

        remaining_stops = self.sorted_stops[self.start_index:]
        remaining_stops.extend(self.sorted_stops[:self.start_index])
        cur_dist = 0

        while len(remaining_stops) != 0:
            for stop in remaining_stops:
                # Start from the smallest remaining angle stop
                cur_stop = stop
                cur_capacity = cur_stop.student_num
                cur_cluster.append(0)
                cur_cluster.append(cur_stop.label)
                cur_dist = self.edges[0].dist_dict[cur_stop.label]
                remaining_stops.remove(stop)
                visted_stop.add(cur_stop.label)

                # select next nearest stop
                next_stop_label = self.next_nearest_stop_label(visted_stop, cur_stop.label)
                next_stop = self.stops_dict.get(next_stop_label)

                if next_stop is None:
                    break

                # check capacity constrain
                while next_stop is not None and cur_capacity + next_stop.student_num <= self.capacity:
                    next_dist = self.edges[cur_stop.label].dist_dict.get(next_stop_label)
                    next2depot_dist = self.edges[0].dist_dict.get(next_stop_label)

                    # check dist constrain
                    if cur_dist + next_dist + next2depot_dist > self.maxing_riding_dist:
                        # use ACS_TSP to check dist constrain again
                        edge_list = cur_cluster.copy()
                        edge_list.append(next_stop_label)
                        # print(edge_list)

                        self.TSP_solver.reset_sub_edges(edge_list)
                        self.TSP_solver.start()

                        if(self.TSP_solver.global_best_distance > self.maxing_riding_dist):
                            break

                    cur_dist += next_dist
                    cur_capacity += next_stop.student_num

                    visted_stop.add(next_stop_label)
                    cur_cluster.append(next_stop_label)

                    remaining_stops.remove(next_stop)
                    cur_stop = next_stop
                    # select next nearest stop
                    next_stop_label = self.next_nearest_stop_label(visted_stop, cur_stop.label)
                    next_stop = self.stops_dict.get(next_stop_label)
                # appen cur cluster and start new cluster
                clusters.append(cur_cluster)
                cur_cluster = []
            if len(remaining_stops) == 0 and len(cur_cluster) != 0:
                clusters.append(cur_cluster)

        return clusters

In [11]:
def sweep_route(pool, acs_list, sweep):

    # cluster first
    cluster = sweep.get_clusters()

    # route second(Use ACS to solve TSP)
    total_dist, routes = mTSP.run(pool, acs_list, cluster)

    return total_dist, routes

def start_mutiple_process_sweep_route(params, ACS_params):
    num_stops = len(depots.index) - 1
    best_route = None
    best_dist = float("inf")
    best_cost = float("inf")
    acs = ACS_TSP.init(trip_matrix, 64, ACS_params)
    acs_list = []
    pool = multiprocessing.Pool(8)

    sweep = Sweep(depots= depots, dist= trip_matrix, capacity = params['capacity'], maxing_riding_dist= params['max_distance'], TSP_solver= acs, start_index = 0)

    for i in range(8):
        acs_list.append(copy.deepcopy(acs))

    # sweep clockwise
    for i in range(num_stops):
        sweep.start_index = i
        cur_dist, cur_route = sweep_route(pool,  acs_list, sweep)
        cur_cost = params['bus_unit_cost'] * len(cur_route) + params['distance_penalty'] * cur_dist
        if cur_cost < best_cost:
            best_dist = cur_dist
            best_route = cur_route
            best_cost = cur_cost

    # sweep counterclockwise
    sweep.reverse_sorted_stops()
    for i in range(num_stops):
        sweep.start_index = i
        cur_dist, cur_route = sweep_route(pool,  acs_list, sweep)
        cur_cost = params['bus_unit_cost'] * len(cur_route) + params['distance_penalty'] * cur_dist
        if cur_cost < best_cost:
            best_dist = cur_dist
            best_route = cur_route
            best_cost = cur_cost

    pool.close()
    return best_dist, best_route, best_cost

### Solution visualization

In [12]:
def convert_to_actual_routes(best_route, trip_matrix):
    actual_best_route = []
    for sub_best_route in best_route:
        num_nodes = len(sub_best_route)
        route = []
        for i in range(num_nodes):
            sub_route = trip_matrix[(trip_matrix['i'] == sub_best_route[i]) & (trip_matrix['j'] == sub_best_route[(i + 1) % num_nodes])]['route'].iloc[0]
            route.append(list(map(int, sub_route[1: -1].split(','))))
        actual_best_route.append(route)
    return actual_best_route

def visualize_solution(trip_matrix, best_route, actual_best_route, graph):
    school = (trip_matrix.iloc[0]['lat'], trip_matrix.iloc[0]['lng'])

    m = folium.Map(location=school, zoom_start=1)
    colors = ['red', 'orange', 'gray', 'green', 'cadetblue', 'blue', 'purple', 'lightblue', 'black', 'lightgreen']
    # prefix = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for route_index in range(len(best_route)):
        sub_best_route = best_route[route_index]
        sub_actual_best_route = actual_best_route[route_index]
        num_nodes = len(sub_best_route)

        for index in range(num_nodes):
            # p1 = [trip_matrix.iloc[sub_best_route[index]]['lat'], trip_matrix.iloc[sub_best_route[index]]['lng']]
            folium.Marker(
                location=[trip_matrix.iloc[sub_best_route[index]]['lat'], trip_matrix.iloc[sub_best_route[index]]['lng']],
                popup="Stop " + str(sub_best_route[index]),
                icon=plugins.BeautifyIcon(
                    icon="arrow-down", icon_shape="circle",
                    number= index,
                    border_color= colors[route_index],
                    background_color='white'
                )
            ).add_to(m)
            # 45-25 26-48 18-55
            # continue if route only contain one node (two stops have the same the nearest node)
            if(len(sub_actual_best_route[index]) == 1): continue
            osmnx.plot_route_folium(graph, sub_actual_best_route[index], route_color=colors[route_index], route_map=m, zoom=1, opacity=0.5, weight=8)

    folium.Marker(
        location=[school[0], school[1]],
        popup="School",
        icon=folium.Icon(color='blue', icon='truck', prefix='fa'),
    ).add_to(m)
    return m

### Solution using Genetic Algorithm

In [13]:
params = {
    "capacity": 72,
    "buses_available": 10,
    "max_distance": 11000,
    "bus_unit_cost": 2000,
    "distance_penalty": 20
}

#### Run the Algorithm

In [14]:
bus_stops = load_bus_stops(depots)
distance_metric = load_distance_metric(trip_matrix)

In [15]:
ga = GA(200,
        0.95,
        0.1,
        0.6,
        0.01,
        bus_stops,
        distance_metric,
        params)
GA_start = time.time()
GA_best_dist, GA_best_route = ga.run_GA(400)
GA_end = time.time()
print('GA_BEST_ROUTES: {}'.format(GA_best_route))
print('GA_BEST_DISTANCE: {}'.format(GA_best_dist))
print("GA time = {} seconds".format(GA_end - GA_start))

GA_BEST_ROUTES: [[0, 29, 58, 62, 13, 53, 42], [0, 21, 63, 60], [0, 37, 38, 27, 25, 44], [0, 16, 17, 55, 34, 33, 18, 19], [0, 39, 40, 22, 49, 50, 51, 61], [0, 47, 48, 36, 57, 12, 1, 2, 56, 54], [0, 20, 3, 4, 52, 9, 10, 11, 31, 32], [0, 41, 14, 15, 59, 28], [0, 8, 5, 6, 7, 35, 30], [0, 24, 23, 43, 45, 26, 46]]
GA_BEST_DISTANCE: 46643.44399999999
GA time = 122.67800641059875 seconds


#### Visualize the results

In [16]:
# Pick midpoint of all coordinates as the center of the graph
midpoint = (depots['lat'].mean(), depots['lng'].mean())

# Calculate radius as distance of farthest coordinate from midpoint
dists = [osmnx.distance.great_circle_vec(midpoint[0], midpoint[1], row['lat'], row['lng']) for _, row in depots.iterrows()]
radius = max(dists) + 2000

# Generate graph
graph = osmnx.graph.graph_from_point(midpoint, dist=radius, clean_periphery=True, simplify=True)

In [17]:
# Convert to actual routes
actual_best_route = convert_to_actual_routes(GA_best_route, trip_matrix)

# Visualize actual routes
visualize_solution(depots, GA_best_route, actual_best_route, graph)

### Solution using Adaptive Genetic Algorithm

#### Run the Algorithm

In [18]:
aga = GA(200,
        0.95,
        0.1,
        0.6,
        0.01,
        bus_stops,
        distance_metric,
        params)
f = exp_schedule(0.6, 0.015, 150)
AGA_start = time.time()
AGA_best_dist, AGA_best_route = aga.run_adaptive_GA(400, f)
AGA_end = time.time()
print('AGA_BEST_ROUTES: {}'.format(AGA_best_route))
print('AGA_BEST_DISTANCE: {}'.format(AGA_best_dist))
print("AGA time = {} seconds".format(AGA_end - AGA_start))

AGA_BEST_ROUTES: [[0, 23, 43, 15, 14, 22, 51, 61, 60, 63, 21], [0, 40, 41, 3, 4, 10, 11], [0, 47, 24, 44, 25, 45, 31, 9, 8, 5, 52], [0, 28, 32, 30, 35, 7, 6, 34, 33, 55], [0, 42, 36, 57, 12, 58, 29, 62, 13], [0, 37, 38, 39, 50, 49, 54, 56, 1, 2, 48, 53], [0, 27, 46, 26, 59, 16, 19, 18, 20, 17]]
AGA_BEST_DISTANCE: 45457.961
AGA time = 138.37900280952454 seconds


#### Visualize the results

In [19]:
# Convert to actual routes
actual_best_route = convert_to_actual_routes(AGA_best_route, trip_matrix)

# Visualize actual routes
visualize_solution(depots, AGA_best_route, actual_best_route, graph)

### Solution using Cluster-First Route-Second (Sweep Nearest Algorithm + Ant Colony System)

In [20]:
ACS_params = {
    "colony_size": 10,
    "alpha": 0.1,
    "beta": 2.0,
    "r0": 0.9,
    "rho": 0.1,
    "pheromone_deposit_weight": 1.0,
    "initial_pheromone": 1.0,
    "iteration": 300
}

#### Run the Algorithm

In [21]:
CFRS_Start = time.time()
CFRS_best_dist, CFRS_best_route, CFRS_best_cost = start_mutiple_process_sweep_route(params, ACS_params)
CFRS_End = time.time()
print('CRFS_BEST_ROUTES: {}'.format(CFRS_best_route))
print('CRFS_BEST_DISTANCE: {}'.format(CFRS_best_dist))
print("CFRS time = {} seconds".format(CFRS_End - CFRS_Start))

CRFS_BEST_ROUTES: [[0, 24, 44, 25, 45, 46, 26, 27, 41], [0, 59, 35, 8, 52, 5, 6, 7, 9, 10, 11, 30, 31, 32], [0, 16, 17, 20, 18, 34, 33, 55, 19], [0, 47, 42, 43, 23, 28, 40, 50, 51, 38, 37, 39], [0, 53, 13, 62, 29, 58, 12, 36, 48], [0, 49, 22, 14, 15, 4, 3, 54, 56, 1, 2, 57], [0, 21, 63, 60, 61]]
CRFS_BEST_DISTANCE: 39745.139
CFRS time = 233.00500559806824 seconds


#### Visualize the results

In [22]:
# Convert to actual routes
actual_best_route = convert_to_actual_routes(CFRS_best_route, trip_matrix)

# Visualize actual routes
visualize_solution(depots, CFRS_best_route, actual_best_route, graph)